# Análisis exploratorio (para el proyecto)

## Antes de partir...

### Requisitos

Usaremos los datos disponibles en [CONASET](https://mapas-conaset.opendata.arcgis.com/).

## Propuesta de proyecto

Esta podría ser una propuesta de proyecto inicial:

* **Situación**: La congestión y el tráfico en las ciudades es un fenómeno natural que emerge de las distintas actividades urbanas y su concentración espacial.
* **Complicación**: El transporte de personas también conlleva accidentes, con la correspondiente pérdida de vidas, problemas de salud y lesiones a las personas involucradas, daño a la propiedad pública y privada, y costo de tiempo a personas no involucradas.
* **Propuesta**: Caracterizar la accidentabilidad en Santiago para apoyar el diseño de políticas públicas que reduzcan la cantidad de accidentes.

Realizaremos un análisis exploratorio para ver su factibilidad e iterar  su definición.

## Bibliotecas necesarias

In [ ]:
# para datos
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from pathlib import Path

# visualización en Python
import seaborn as sns
import matplotlib.pyplot as plt

from chiricoca.config import setup_style

setup_style()

## Construcción del dataset

Construir el dataset también involucra descargarlo y almacenarlo.

Lo descargamos [aquí](https://mapas-conaset.opendata.arcgis.com/search?categories=%252Fcategories%252Fregiones-%252Fregi%25C3%25B3n%2520metropolitana%2520de%2520santiago&groupIds=fca1f61c6556499db843c09cc80c70c0), o con el código que está más abajo ;)

In [ ]:
data_path = Path("data") / "conaset"

if not data_path.exists():
    data_path.mkdir(parents=True)

In [ ]:
urls = {
    '2019': 'https://hub.arcgis.com/api/v3/datasets/58cbd7c35ad14cfd955b1afa3a2b62fd_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1',
    '2020': 'https://hub.arcgis.com/api/v3/datasets/213520f62eb443dd9bdc3398c617a811_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1',
    '2021': 'https://hub.arcgis.com/api/v3/datasets/4b636b2f27164b0ebbeca0ab9db4d08a_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1',
    '2022': 'https://hub.arcgis.com/api/v3/datasets/88dd8179f726431da91f3ef4847a8dd2_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1',
    '2023': 'https://hub.arcgis.com/api/v3/datasets/30467ae1a8ef4708b64a4273b3672138_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1',
}

files = {}

for year, url in urls.items():
    file_path = data_path / f"{year}-rm-car-accidents.json"
    files[year] = file_path
    if not file_path.exists():
        r = requests.get(url)
        with open(file_path, 'wt') as f:
            f.write(r.text)

In [ ]:
gdf = {year: gpd.read_file(x, dtypes={'Fecha': 'str', 'Hora': 'str'}).to_crs('epsg:5361') for year, x in files.items()}
gdf

## Limpiado, Filtrado y Pre-Procesamiento

In [ ]:
fig, axes = plt.subplots(1, 5)

for ax, year in zip(axes, files):
    gdf[year].plot(ax=ax, marker='.', markersize=1, alpha=0.01)
    ax.set_aspect('equal')
    ax.set_title(year)


In [ ]:
#gdf['2019'].groupby('Fecha').size().plot()
#gdf['2019'].rolling(14, on='Fecha').mean()

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(6, 12))

for ax, year in zip(axes, files):
    try:
        gdf[year].groupby('Fecha').size().plot(ax=ax)
    except Exception as e:
        pass
    ax.set_title(year)
    ax.set_xlabel('')

In [ ]:
gdf['2022'][['Fecha', 'Hora']]

In [ ]:
gdf['2022']['Fecha'].describe()

In [ ]:
gdf['2022']['Fecha'].max() - gdf['2022']['Fecha'].min()

In [ ]:
gdf['2022']['Fecha'].head().map(lambda x: pd.Timedelta(x, unit='days')) + pd.to_datetime('1900-01-01')

In [ ]:
gdf['2022']['Fecha_original'] = gdf['2022']['Fecha']

In [ ]:
gdf["2022"]["Fecha"] = gdf["2022"]["Fecha_original"].map(
    lambda x: pd.Timedelta(x - 2, unit="days")
) + pd.to_datetime("1900-01-01", utc=True)
gdf['2022']['Fecha'].describe()

In [ ]:
fig, axes = plt.subplots(5, 1)

for ax, year in zip(axes, files):
    try:
        gdf[year].groupby('Fecha').size().plot(ax=ax)
    except Exception as e:
        pass
    ax.set_title(year)
    ax.set_xlabel('')

In [ ]:
for year, values in gdf.items():
    result = pd.isna(values).astype(int).describe().loc['mean']
    print(result)

In [ ]:
accidents = pd.concat(gdf.values())

In [ ]:
accidents.groupby('Fecha').size().plot()

In [ ]:
null_test = pd.isna(accidents).astype(int).describe().loc['mean']
fig, ax = plt.subplots(figsize=(6, 9))
null_test[null_test > 0].sort_values().plot(kind='barh', linewidth=1, width=0.9)

In [ ]:
accidents.filter(like='Cau').sample(10)

In [ ]:
len(accidents)

In [ ]:
accidents['Causa'].describe()

In [ ]:
accidents['Causa__CON'].describe()

In [ ]:
accidents.filter(like='Ubi').sample(10)

In [ ]:
accidents['Ubicación'].describe()

In [ ]:
accidents.filter(like='TIPO').sample(10)

In [ ]:
accidents.filter(like='Tip').sample(10)

In [ ]:
accidents['Tipo__CONA'].describe()

In [ ]:
accidents['Tipo_Accid'].describe()

In [ ]:
accidents.columns

In [ ]:
len(accidents)
for values in gdf.values():
    print(len(values))

In [ ]:
fig, ax = plt.subplots()

sns.heatmap(
    accidents[["Fallecidos", "Graves", "Menos_Grav", "Leves", "Ilesos", "Lesionados"]],
    ax=ax,
    yticklabels=False,
)

In [ ]:
fig, ax = plt.subplots()

sns.heatmap(
    accidents[["Fallecidos", "Graves", "Menos_Grav", "Leves", "Ilesos", "Lesionados"]],
    ax=ax,
    yticklabels=False,
)

ymin = 0
props = dict(
    connectionstyle="angle, angleA=90, angleB=180, rad=0",
    arrowstyle="-",
    shrinkA=10,
    shrinkB=10,
    color="black",
    lw=1,
)

for year, values in gdf.items():
    ymax = ymin + len(values)
    x = 0
    dy = len(values)
    dx = -0.5

    ax.annotate(
        year,
        xy=(x, ymin),
        xytext=(x + dx, ymin + dy / 2),
        annotation_clip=False,
        arrowprops=props,
    )
    ax.annotate(
        year,
        xy=(x, ymax),
        xytext=(x + dx, ymin + dy / 2),
        annotation_clip=False,
        alpha=0,
        arrowprops=props,
    )

    ymin = ymax
    # break

In [ ]:
sns.heatmap(pd.notnull(accidents[['Comuna', 'Comuna_1', 'COMUNAREAL']]))

In [ ]:
accidents['comuna'] = accidents['Comuna']
accidents.loc[pd.isnull(accidents['comuna']),'comuna'] = accidents['COMUNAREAL'][pd.isnull(accidents['comuna'])]
accidents.loc[pd.isnull(accidents['comuna']),'comuna'] = accidents['Comuna_1'][pd.isnull(accidents['comuna'])]
accidents['comuna'].describe()

## Filtrar

En http://bboxfinder.com podemos definir una _caja contenedora_ (_bounding box_) para área de análisis.\
CRS = Coordinate Reference System

In [ ]:
from chiricoca.geo.utils import clip_point_geodataframe

In [ ]:
scl_bounds = [-70.88006218, -33.67612715, -70.43015094, -33.31069169]

In [ ]:
scl = clip_point_geodataframe(accidents.to_crs('epsg:4326'), scl_bounds).to_crs(accidents.crs)
scl.plot(marker='.', markersize=1, column='comuna')

In [ ]:
scl['month'] = scl['Fecha'].dt.month
scl['year'] = scl['Fecha'].dt.year
scl['day_of_week'] = scl['Fecha'].dt.dayofweek

In [ ]:
len(scl)

## ¿Qué contiene?

In [ ]:
scl["Tipo_Accid"].value_counts().sort_values(ascending=True).plot(
    kind="barh", linewidth=0, width=0.9
)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

scl["Causa"].value_counts().sort_values(ascending=True).plot(
    kind="barh", linewidth=0, width=0.9, ax=ax, logx=True
)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

scl["Causa__CON"].value_counts().sort_values(ascending=True).plot(
    kind="barh", linewidth=0, width=0.9, ax=ax, logx=True
)

In [ ]:
victimas = scl[["Fallecidos", "Graves", "Menos_Grav", "Leves"]].assign(
    total=lambda x: x.sum(axis=1)
).groupby("total").sum().drop(0)

victimas

In [ ]:
victimas.sum(axis=1).plot(kind='bar', width=0.95, linewidth=0)

In [ ]:
victimas.plot(
    kind="bar", logy=True, linewidth=0, width=0.9
)

In [ ]:
from chiricoca.base.weights import normalize_rows
victimas.pipe(normalize_rows)

In [ ]:
from chiricoca.base.weights import normalize_rows

fig, ax = plt.subplots()

victimas.pipe(normalize_rows).plot(
    kind="bar", stacked=True, linewidth=0, width=0.9, ax=ax
)

ax.set_ylim([0,1])

## ¿Cuándo?

In [ ]:
fig, ax = plt.subplots(figsize=(9, 2))
scl.resample('1W', on='Fecha').size().plot(linewidth=0.5)

In [ ]:
scl.groupby(['year', 'day_of_week']).size().unstack().pipe(normalize_rows)

In [ ]:
sns.heatmap(scl.groupby(['year', 'day_of_week']).size().unstack().pipe(normalize_rows), annot=True, linewidth=1, fmt='.4f')

## ¿Dónde?

In [ ]:
scl['victimas'] = scl[["Fallecidos", "Graves", "Menos_Grav"]].sum(axis=1)

In [ ]:
from chiricoca.maps import heat_map
from chiricoca.geo.figures import small_multiples_from_geodataframe

fig, axes = small_multiples_from_geodataframe(scl, 2)

heat_map(scl, bandwidth=0.005, low_threshold=0.005, ax=axes[0])
heat_map(scl, weight="victimas", bandwidth=0.005, low_threshold=0.005, ax=axes[1])

## ¿Cómo?

In [ ]:
tipo_por_comuna = (
    scl.groupby(["comuna", "Tipo__CONA"]).size()
    .unstack(fill_value=0)
)

tipo_por_comuna


In [ ]:
from chiricoca.tables import barchart

barchart(
    tipo_por_comuna,
    stacked=True,
    sort_items=True,
    normalize=True,
    horizontal=True,
    palette="cubehelix"
)

In [ ]:
sns.heatmap(
    scl.groupby(["Causa__CON", "day_of_week"])
    .size()
    .unstack(fill_value=0)
    .pipe(normalize_rows), annot=True, fmt='.4f', linewidth=1, cbar=False
)

In [ ]:
sns.pointplot(scl, x='month', y='victimas')

In [ ]:
sns.pointplot(scl[scl['victimas'] > 0], x='month', y='victimas', hue='year', dodge=True)

In [ ]:
scl.groupby("Causa__CON")["victimas"].mean().sort_values(ascending=True).plot(
     kind="barh", cmap="viridis", linewidth=0, width=0.9
)

In [ ]:
#sns.boxplot(scl, x='victimas', y='Causa__CON')

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(9, 9))

mean_accidents = (
    lambda name, x: x.resample("1d", on="Fecha")
    .size()
    # ["victimas"].sum()
    .rolling(28, center=True)
    .mean()
    .rename(name)
    .to_frame()
    .apply(lambda x: (x - x.mean()) / x.std())
)

for ax, cat in zip(axes, ["COLISION", "CHOQUE", "ATROPELLO"]):
    # print(g)
    g = scl[scl["Tipo__CONA"] == cat]
    mean_accidents("all", scl).plot(ax=ax, color="grey", linewidth=0.5, legend=False)
    mean_accidents(cat, g).plot(ax=ax, linewidth=1.5, color="purple", legend=False)
    ax.set_title(cat)
    sns.despine(ax=ax, left=True, bottom=True)

fig.tight_layout()

# Entonces...

¿Es **factible** hacer algo con estos datos? Sí, aunque pareciera que se debe hacer algo a nivel general. La granularidad espacial es buena, pero temporalmente hay inconsistencias graves. Espacialmente también. Además hay pocos meta-datos de los accidentes relacionados con las condiciones en las que sucedieron, por ej., no se incluye el vehículo o las características de las personas.

Quizás lo que más llama la atención es la variación en la proporción de accidentes por día de semana. Ya que tenemos variabilidad espacial y en la proporción de accidentes por día de semana (o bien semana y fin de semana), podríamos iterar nuestro proyecto para que el objetivo sea aportar evidencia para:

1. Desarrollar estrategias de reducción de imprudencia de conductores. La tarea a resolver es **encontrar la relación** entre ubicación, accidentes e infraestructura del lugar. Posiblemente se requieran datos adicionales sobre el entorno construido;
2. Intensificar controles de alcoholemia los fines de semana. La tarea a resolver es **identificar** puntos críticos de accidentes en tipos de días específicos.
   
¡En estos casos, el proyecto parece factible!

¿Propones otros análisis futuros?